In [1]:
import os
import base64
import tensorflow as tf
from googleapiclient import discovery
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
TFR_TEST = 'test.tfrecord'
BUCKET = 'gs://robolab/'

NUM_CLASSES = 2

IMG_HEIGHT = 80
IMG_WIDTH = 71

In [3]:
def predict_json(project, model, key, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    
    scopes = ['https://www.googleapis.com/auth/cloud-platform']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(key, scopes=scopes)
    service = discovery.build('ml', 'v1', credentials=credentials)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [4]:
def get_image_id(serialized_example):

    features = tf.parse_single_example(
        serialized_example,
        features={'image_raw': tf.FixedLenFeature([], tf.string),
                  'image_id': tf.FixedLenFeature([], tf.string)})
    
    return features['image_id']

In [5]:
preds = []
image_id_tensors = []

record_iterator = tf.python_io.tf_record_iterator(path=BUCKET + TFR_TEST)

for string_record in record_iterator:

    image_id_tensors.append(get_image_id(string_record))
    
    data = base64.b64encode(string_record).decode('utf-8')
    
    preds.extend(predict_json(
        'psyched-timer-223917',
        'subvoc',
        'private_key.json',
        [{"b64": data}],
        version=None))
    
    break

In [6]:
with tf.Session() as sess:
     image_ids = sess.run(image_id_tensors)

In [7]:
for image_id, pred in zip(image_ids, preds):
    image_id = image_id.decode('utf-8')
    print('id: {}, label: {}, probs: [{:.3f}, {:.3f}]'.format(
        image_id, pred['classes'], pred['probabilities'][0], pred['probabilities'][1]))

id: ZM5JX5MM, label: 1, probs: [0.012, 0.988]
